In [1]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:

df = pd.read_json(
    "the-pudding-pockets/measurementRectangles.json"
)  # to not index on user id..as we got duplicates that we need
print(df.head())
print(df.shape)

               brand     style menWomen             name  \
0  7 for All Mankind  straight      men     The Straight   
1  7 for All Mankind    skinny      men           Paxtyn   
2        Abercrombie      slim      men     Langdon Slim   
3        Abercrombie  straight      men  Kennan Straight   
4     American Eagle  straight      men         Straight   

                                   fabric   price  maxHeightFront  \
0                  98% cotton, 2% spandex 179.000          21.500   
1    92% cotton, 6% polyester, 2% spandex 209.000          21.000   
2   95% cotton, 4% polyester, 1% elastane  78.000          22.000   
3   95% cotton, 4% polyester, 1% elastane  78.000          22.500   
4  85% cotton, 13% polyester, 2% elastane  49.950          22.000   

   minHeightFront  rivetHeightFront  maxWidthFront  ...  waistSize  \
0          23.000             7.000         16.500  ...          0   
1          23.000             7.500         16.500  ...          0   
2          15.

In [3]:
df.describe()

,price,maxHeightFront,minHeightFront,rivetHeightFront,maxWidthFront,minWidthFront,maxHeightBack,minHeightBack,maxWidthBack,minWidthBack,waistSize,pocketArea
count,80.000,80.000,80.000,80.000,80.000,80.000,80.000,80.000,80.000,80.000,80.000,80.000
mean,80.750,18.729,15.654,6.654,15.777,12.810,15.565,13.026,13.521,11.949,0.000,8424.815
std,44.552,4.887,3.504,0.960,1.470,1.049,0.922,0.949,0.865,0.890,0.000,2699.333
min,9.990,11.500,9.500,4.500,12.000,11.000,13.000,10.500,11.500,9.500,0.000,4454.429
25%,49.950,14.000,13.000,6.000,14.500,12.000,15.000,12.425,13.000,11.500,0.000,5905.184
50%,73.975,20.250,15.000,6.500,16.000,12.600,15.500,13.000,13.500,12.000,0.000,8618.737
75%,95.713,22.500,17.000,7.000,17.000,13.500,16.050,13.700,14.000,12.500,0.000,10724.592
max,249.000,28.000,25.000,9.200,19.000,16.000,17.500,15.000,15.500,14.000,0.000,13102.032


## Assumptions -> can become tests

- women jeans are more expensive
- women pockets are smaller
- women jeans that are more expensive have smaller pockets -> you should wear a bag with if ur jeans are expensive
- create data file with some missing values - > so we can do imputation , for example prices.. get average, or 0 or average per gender

- colums I would add: is_size_smaller_than_average ( per brand / per dataset )



- method: get_diversity_score -> return a score based on if women pocket sizes are higher than average... by brand in relation to the rest maybe
- homework: write the test